In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

In [2]:
test_df = pd.read_csv('./dataset/Test_dataset.csv')
#test_df = test_df[['uuid','X_LOC', 'Y_LOC', 'DEPT', 'NPHI','DTC', 'RHOB', 'GR','CALI']]
test_df

,uuid,file_name,ROPA,BS,PEF,RXO,DCAL,RMIC,ROP,SP,...,DTS,RHOB,NPHI,SGR,X_LOC,CALI,RSHA,Z_LOC,RDEP,DEPT
0,14812,25_10-10,53.066914,12.250001,3.125396,NaN,0.051304,NaN,45.224689,NaN,...,NaN,2.131319,0.398266,NaN,459853.34375,12.301304,0.985707,-1308.193359,1.019758,1348.310400
1,72908,25_10-10,53.109482,12.250001,3.292384,NaN,0.049227,NaN,49.948738,NaN,...,NaN,2.129777,0.389460,NaN,459853.34375,12.299226,0.888855,-1308.345337,0.924186,1348.462400
2,103104,25_10-10,53.152004,12.250001,3.693287,NaN,0.047644,NaN,52.813866,NaN,...,NaN,2.138082,0.394868,NaN,459853.34375,12.297644,0.870774,-1308.497314,0.895908,1348.614400
3,23480,25_10-10,53.194557,12.250001,4.051715,NaN,0.026056,NaN,58.635353,NaN,...,NaN,2.153999,0.389355,NaN,459853.34375,12.276056,0.913030,-1308.649292,0.955895,1348.766400
4,104267,25_10-10,53.237164,12.250001,4.299806,NaN,0.049645,NaN,58.242565,NaN,...,NaN,2.140920,0.365808,NaN,459853.34375,12.299644,0.959649,-1308.801392,1.022451,1348.918400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,34_3-3 A,12.779722,8.500000,8.879548,NaN,NaN,NaN,15.756256,NaN,...,NaN,2.515146,NaN,NaN,NaN,8.647468,NaN,NaN,NaN,5006.809976
121793,26727,34_3-3 A,11.651907,8.500000,10.237462,NaN,NaN,NaN,15.760216,NaN,...,NaN,2.543073,NaN,NaN,NaN,8.636636,NaN,NaN,NaN,5006.961976
121794,83995,34_3-3 A,11.266949,8.500000,10.901587,NaN,NaN,NaN,15.990236,NaN,...,NaN,2.576337,NaN,NaN,NaN,8.592650,NaN,NaN,NaN,5007.113976
121795,67375,34_3-3 A,11.116782,8.500000,10.468700,NaN,NaN,NaN,15.314774,NaN,...,NaN,2.606787,NaN,NaN,NaN,8.546233,NaN,NaN,NaN,5007.265976


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121797 entries, 0 to 121796
Data columns (total 27 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   uuid       121797 non-null  int64  
 1   file_name  121797 non-null  object 
 2   ROPA       30066 non-null   float64
 3   BS         89910 non-null   float64
 4   PEF        86459 non-null   float64
 5   RXO        6707 non-null    float64
 6   DCAL       41164 non-null   float64
 7   RMIC       14930 non-null   float64
 8   ROP        87913 non-null   float64
 9   SP         75533 non-null   float64
 10  RMED       107922 non-null  float64
 11  GR         121794 non-null  float64
 12  DTC        120938 non-null  float64
 13  MUDWEIGHT  33364 non-null   float64
 14  Y_LOC      114368 non-null  float64
 15  DRHO       95297 non-null   float64
 16  DEPTH_MD   114368 non-null  float64
 17  DTS        25344 non-null   float64
 18  RHOB       96178 non-null   float64
 19  NPHI       78221 non-nu

In [ ]:
df = pd.read_csv('dataset/estimate_test_dataset.csv')
df

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Definisi hubungan fitur untuk imputasi
imputation_dict = {
    'DTC': ['RHOB', 'DEPT'],
    'RHOB': ['DTC', 'DEPT'],
}

def impute_feature(df, target_feature, predictor_features):
    # Periksa apakah kolom target 100% NaN
    if df[target_feature].isna().sum() == len(df):
        print(f"Skipping {target_feature}: 100% NaN.")
        return df  # Kembalikan DataFrame tanpa perubahan
    
    # Periksa apakah semua kolom prediktor 100% NaN
    valid_predictors = [col for col in predictor_features if df[col].isna().sum() < len(df)]
    
    if len(valid_predictors) == 0:
        print(f"Skipping {target_feature}: All predictor columns are 100% NaN.")
        return df  # Kembalikan DataFrame tanpa perubahan

    # Cek apakah ada NaN yang perlu diimputasi di target
    if df[target_feature].isna().sum() > 0:
        print(f"Imputing {target_feature} using {valid_predictors}...")

        # Ambil subset data dengan fitur prediktor valid dan target
        imputation_data = df[valid_predictors + [target_feature]].copy()

        # Standarisasi data
        scaler = StandardScaler()
        imputation_data_scaled = scaler.fit_transform(imputation_data)

        # Terapkan KNN Imputer
        imputer = KNNImputer(n_neighbors=5, weights='distance')
        imputed_data_scaled = imputer.fit_transform(imputation_data_scaled)

        # Kembalikan ke skala asli
        imputed_data = scaler.inverse_transform(imputed_data_scaled)
        imputed_df = pd.DataFrame(imputed_data, columns=valid_predictors + [target_feature], index=df.index)

        # Update DataFrame asli
        df[target_feature] = imputed_df[target_feature]
    else:
        print(f"No NaN values in {target_feature}. Skipping imputation.")
    
    return df

In [ ]:
# Create a copy of test_df to work with
#df = test_df.copy()

# Lakukan imputasi untuk setiap fitur target
for target_feature, predictor_features in imputation_dict.items():
    # Pastikan semua fitur prediktor ada di DataFrame
    if all(feature in df.columns for feature in predictor_features):
        df = impute_feature(df, target_feature, predictor_features)
    else:
        print(f"Skipping imputation for {target_feature}")

In [ ]:
from sklearn.linear_model import LinearRegression

train_df = df.dropna(subset=['NPHI'])
test_df = df[df['NPHI'].isna()]

if not test_df.empty:
    model = LinearRegression()
    predictors = ['RHOB', 'GR', 'DTC', 'CALI']
    model.fit(train_df[predictors], train_df['NPHI'])
    df.loc[df['NPHI'].isna(), 'NPHI'] = model.predict(test_df[predictors])

In [ ]:
from scipy.spatial import cKDTree
import numpy as np

# Ensure X_LOC and Y_LOC have no missing values
df = df.dropna(subset=['X_LOC', 'Y_LOC'])

# Create KDTree using valid spatial coordinates
coords = df[['X_LOC', 'Y_LOC']].values
tree = cKDTree(coords)

# Find the 2 nearest neighbors (excluding itself)
_, idx = tree.query(coords, k=3)  # k=3 for backup if the first neighbor is missing

# Fill missing values using nearest neighbor
for col in ['NPHI', 'DTC', 'SP', 'GR', 'RHOB', 'CALI']:
    missing_mask = df[col].isna()  # Find missing values
    
    # Create an empty array to store filled values
    filled_values = df[col].values.copy()
    
    for i, is_missing in enumerate(missing_mask):
        if is_missing:
            # Check nearest neighbors
            for neighbor_idx in idx[i, 1:]:  # Skip self (idx[i, 0] is the same point)
                if not np.isnan(df.iloc[neighbor_idx][col]):  # Use first non-NaN neighbor
                    filled_values[i] = df.iloc[neighbor_idx][col]
                    break  # Stop after finding the first valid neighbor
    
    # Assign the new filled values
    df[col] = filled_values

In [ ]:
df.drop(columns='SP', inplace=True)
df.fillna(0,inplace=True)
df

In [ ]:
df.to_csv('result_cleaning2.csv', index=False)

In [1]:
from pycaret.classification import load_model
from pycaret.classification import *

# Muat kembali model
loaded_model = load_model('best_model')

# Gunakan model yang telah dimuat kembali untuk prediksi
test_predictions = predict_model(loaded_model, data=df)

ModuleNotFoundError: No module named 'pycaret'

In [22]:
test_predictions

,uuid,X_LOC,Y_LOC,DEPT,NPHI,DTC,RHOB,GR,CALI,prediction_label,prediction_score
0,14812,459853.34375,6560993.0,1348.310425,0.398266,133.537033,2.131319,61.177399,12.301304,30000,0.34
1,72908,459853.34375,6560993.0,1348.462402,0.389460,133.525543,2.129777,63.512333,12.299226,30000,0.37
2,103104,459853.34375,6560993.0,1348.614380,0.394868,130.739624,2.138082,63.515835,12.297644,30000,0.39
3,23480,459853.34375,6560993.0,1348.766357,0.389355,128.074249,2.153999,63.153057,12.276056,30000,0.36
4,104267,459853.34375,6560993.0,1348.918457,0.365808,121.454926,2.140920,60.224148,12.299644,30000,0.31
...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,0.00000,0.0,5006.810059,0.231205,80.453865,2.515146,65.925987,8.647468,65030,0.50
121793,26727,0.00000,0.0,5006.961914,0.232819,80.927139,2.543073,66.297127,8.636636,65030,0.48
121794,83995,0.00000,0.0,5007.113770,0.260863,80.394623,2.576337,64.853714,8.592650,65030,0.50
121795,67375,0.00000,0.0,5007.266113,0.212640,83.122765,2.606787,62.779541,8.546233,65030,0.49


In [24]:
import pandas as pd

# Data mapping berdasarkan label_code dan label_name
lithology_mapping = {
    30000: "Sandstone",
    65030: "Sandstone/Shale",
    65000: "Shale",
    80000: "Marl",
    74000: "Dolomite",
    70000: "Limestone",
    70032: "Chalk",
    88000: "Halite",
    86000: "Anhydrite",
    99000: "Tuff",
    90000: "Coal",
    93000: "Basement"
}


# Buat kolom baru berdasarkan mapping
test_predictions['label_name'] = test_predictions['prediction_label'].map(lithology_mapping)

test_predictions

,uuid,X_LOC,Y_LOC,DEPT,NPHI,DTC,RHOB,GR,CALI,prediction_label,prediction_score,lithology_name,label_name
0,14812,459853.34375,6560993.0,1348.310425,0.398266,133.537033,2.131319,61.177399,12.301304,30000,0.34,Sandstone,Sandstone
1,72908,459853.34375,6560993.0,1348.462402,0.389460,133.525543,2.129777,63.512333,12.299226,30000,0.37,Sandstone,Sandstone
2,103104,459853.34375,6560993.0,1348.614380,0.394868,130.739624,2.138082,63.515835,12.297644,30000,0.39,Sandstone,Sandstone
3,23480,459853.34375,6560993.0,1348.766357,0.389355,128.074249,2.153999,63.153057,12.276056,30000,0.36,Sandstone,Sandstone
4,104267,459853.34375,6560993.0,1348.918457,0.365808,121.454926,2.140920,60.224148,12.299644,30000,0.31,Sandstone,Sandstone
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121792,10928,0.00000,0.0,5006.810059,0.231205,80.453865,2.515146,65.925987,8.647468,65030,0.50,Sandstone/Shale,Sandstone/Shale
121793,26727,0.00000,0.0,5006.961914,0.232819,80.927139,2.543073,66.297127,8.636636,65030,0.48,Sandstone/Shale,Sandstone/Shale
121794,83995,0.00000,0.0,5007.113770,0.260863,80.394623,2.576337,64.853714,8.592650,65030,0.50,Sandstone/Shale,Sandstone/Shale
121795,67375,0.00000,0.0,5007.266113,0.212640,83.122765,2.606787,62.779541,8.546233,65030,0.49,Sandstone/Shale,Sandstone/Shale


In [25]:
submission = test_predictions[['uuid','label_name']]

In [26]:
submission.to_csv("submission.csv",index=False)